In [42]:
!pip install openai transformers datasets rouge_score nltk numpy torch flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash_attn


In [2]:
from huggingface_hub import login
import torch
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer
import re
import json
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import numpy as np

torch.cuda.empty_cache()
# Log in with access token
# to be replaced
login(token="hf_you_hf_token_here")

Access: ZHcfFFsrftNHdCIGXgzyRosnfjZOEGNlYz

- load dataset

In [3]:
dataset = load_dataset("sobamchan/aclsum", "extractive")["test"]
val_dataset = load_dataset("sobamchan/aclsum", "extractive")["validation"]
print(dataset[0].keys())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'])


- helper function to load the correct model

In [4]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# def load_model_and_tokenizer(model_name: str):
#     if "Qwen" in model_name:
#         # Qwen models require trust_remote_code=True
#         tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#         model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto")
#         print("Loaded Qwen model and tokenizer.")
#     elif "Llama" in model_name:
#         # Llama models do not require trust_remote_code
#         tokenizer = AutoTokenizer.from_pretrained(model_name)
#         model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
#         print("Loaded Llama model and tokenizer.")
#     else:
#         raise ValueError("Unsupported model name. Please use a Qwen or Llama model.")

#     if tokenizer.pad_token_id is None:
#         tokenizer.pad_token = tokenizer.eos_token

#     return model, tokenizer

In [5]:
print(dataset)

Dataset({
    features: ['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'],
    num_rows: 100
})


In [6]:
print(dataset['challenge_labels'][0])

[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


- create correct gold standard

In [7]:
# Aspects to analyze
ASPECTS = ["challenge", "approach", "outcome"]

def labels_to_indices(labels):
    return [i+1 for i, v in enumerate(labels) if int(v) == 1]

def gold_for_doc(example):
    return {
        "challenge": labels_to_indices(example["challenge_labels"]),
        "approach":  labels_to_indices(example["approach_labels"]),
        "outcome":   labels_to_indices(example["outcome_labels"]),
    }

def gold_for_dataset(ds):
    return [gold_for_doc(ex) for ex in ds]

def indices_to_text(sentences, idxs_1b):
    return " ".join(sentences[i-1] for i in idxs_1b if 1 <= i <= len(sentences))

gold_all = gold_for_dataset(dataset)
print(gold_all[0])

{'challenge': [1, 2, 7, 11], 'approach': [3, 15, 19, 26, 27], 'outcome': [4, 5, 20, 21, 30, 31]}


- capping to try to improve predictions - this avoids overprediction for the model and it will drasticly improve precision. What was notices is that consistently, both models regardless of prompting technique tend to predict a number of indices which is much larger than the gold standard. Capping implies using the median gold sentences per aspect to restrict the number of predicted indices. This is what the following functions do.
- all prompting techniques have both the capped and not capped version

In [8]:
def aspect_caps_from_gold(gold_all, aspects=ASPECTS, min_cap=1):
    """
    Derive a sensible per-aspect cap K using the median #gold sentences per aspect.
    Ensures at least `min_cap` (default 1).
    Returns: dict like {"challenge": 4, "approach": 5, "outcome": 5}
    """
    counts = {a: [] for a in aspects}
    for g in gold_all:
        for a in aspects:
            counts[a].append(len(g.get(a, [])))
    return {a: max(min_cap, int(np.median(counts[a]) or 0)) for a in aspects}

# Build once after you computed `gold_all`
CAPS = aspect_caps_from_gold(gold_all)
print("Caps:", CAPS)

Caps: {'challenge': 4, 'approach': 6, 'outcome': 4}


In [9]:
def _cap_indices_in_order(indices, cap):
    """Keep at most `cap` indices, preserving ascending order."""
    if cap is None or cap <= 0:
        return indices
    return indices[:cap]

In [10]:
def make_capped_renderer(base_renderer, caps_dict):
    """
    base_renderer: a function like vanilla_prompt(sentences, target_label)
    caps_dict: e.g., CAPS from step 1
    Returns a new renderer with an extra constraint appended to the prompt.
    """
    def capped_renderer(sentences, target_label):
        cap = caps_dict.get(target_label)
        prompt = base_renderer(sentences, target_label)
        # Append a crisp constraint the model will see
        return (
            prompt
            + f"\n\nAdditional constraint:\n"
              f"- Select AT MOST {cap} sentences for the \"{target_label}\" aspect."
        )
    return capped_renderer


- prompting techniques

Vanilla prompt is the most basic version of the prompt and it is constructed as indicated by the instructions

In [11]:
# vanilla prompt
def vanilla_prompt(sentences, target_label):
    # target_label ∈ {"challenge","approach","outcome"}
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)
    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Rules:
- Select ONLY sentences that primarily express the "{target_label}" aspect.
- If none match, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [12]:
vanilla_capped = make_capped_renderer(vanilla_prompt, CAPS)

**least to most**: The document is broken into thematic sections (e.g., problem, method, results), and representative sentences are extracted from each.

All more complex prompting techniques are also brought in their one shot version, which takes a random instance from the validation dataset (with set random seed for reproducibility) and uses it as an example

In [13]:
def least_to_most_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

First, consider the overall purpose of the document and how each sentence contributes to it.
Then, from that understanding, select ONLY sentences that primarily express the "{target_label}" aspect.
If none match, return an empty list.
Indices are 1-based.
Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [14]:
least_to_most_capped = make_capped_renderer(least_to_most_prompt, CAPS)

In [15]:
import random

def least_to_most_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42, k=1):
    random.seed(seed)
    examples = random.sample(list(val_dataset), k)

    # Format each example
    example_blocks = []
    for ex in examples:
        ex_sents = ex["source_sentences"]
        # Which labels to use
        gold_indices = [i+1 for i, lab in enumerate(ex[f"{target_label}_labels"]) if lab == 1]
        numbered_ex = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
        block = f'''Input:
{"\n".join(numbered_ex)}

Selected indices: {gold_indices}'''
        example_blocks.append(block)

    # The "new task" input
    numbered_input = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example(s) (from validation set):
{"\n\n".join(example_blocks)}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1. First, consider the overall purpose of the document and how each sentence contributes to it.
2. Then, select ONLY sentences that primarily express the "{target_label}" aspect.
3. If none match, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON. Do not include explanations or reasoning in your final output.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [16]:
least_to_most_one_shot_capped = make_capped_renderer(least_to_most_one_shot, CAPS)

**tool augmented**: Each sentence is accompanied by a precomputed importance score (e.g., TF-IDF, BERTScore, LexRank), guiding selection.

In [17]:
def tool_augmented_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. For each sentence, use the internal `check_aspect(sentence, aspect)` tool.
2. The tool's output is 'match' if the sentence primarily describes the "{target_label}" aspect, otherwise it is 'no_match'.
3. List the sentences that result in a 'match'.
4. If no sentences match, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [18]:
tool_augmented_capped = make_capped_renderer(tool_augmented_prompt, CAPS)

In [19]:
def tool_augmented_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42):
    assert target_label in ASPECTS, f"target_label must be one of {ASPECTS}"

    rng = random.Random(seed)

    # --- pick a single validation example that has at least one positive for this aspect ---
    idxs = list(range(len(val_dataset)))
    rng.shuffle(idxs)
    chosen = None
    for i in idxs:
        ex = val_dataset[i]
        labs = ex[f"{target_label}_labels"]
        if any(int(v) == 1 for v in labs):
            chosen = ex
            break
    if chosen is None:
        # fallback: just take the first one
        chosen = val_dataset[0]

    ex_sents = chosen["source_sentences"]
    ex_labs  = [int(v) for v in chosen[f"{target_label}_labels"]]
    ex_gold  = [j+1 for j, v in enumerate(ex_labs) if v == 1]

    # --- format the example input as numbered sentences ---
    numbered_example = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
    # Create a synthetic "tool trace" where the tool says 'match' iff gold==1
    tool_trace_lines = []
    for i, s in enumerate(ex_sents):
        verdict = "match" if ex_labs[i] == 1 else "no_match"
        # keep the sentence text in the trace to model the behavior (can be long; OK for one-shot)
        tool_trace_lines.append(
            f'check_aspect("Sentence {i+1}: {s}", "{target_label}") -> "{verdict}"'
        )

    # --- the new task's numbered input ---
    numbered_input = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    # --- assemble the prompt ---
    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example (one-shot from the validation set; fixed seed):
Input:
{"\n".join(numbered_example)}

Thinking Process (Tool Use; do not include in final output):
For each sentence, call the internal tool:
{chr(10).join(tool_trace_lines)}

From the tool results above, list the indices that returned "match".
Selected indices:
{ex_gold}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1) For each sentence, use the internal `check_aspect(sentence, aspect)` tool.
2) The tool returns "match" if the sentence primarily expresses the "{target_label}" aspect; otherwise "no_match".
3) Collect the indices (1-based) of all sentences that return "match".
4) If none match, return an empty list.
5) Return EXACTLY one JSON object on a single line, with no extra text.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [20]:
tool_augmented_one_shot_capped = make_capped_renderer(tool_augmented_one_shot, CAPS)

**scoring based**: The model assigns scores to each sentence (e.g., 1–5 or 1–10) and selects those with the highest ratings.

In [21]:
def scoring_based_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. For each sentence, assign a score from 1 (low relevance) to 5 (high relevance) for how well it expresses the "{target_label}" aspect.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [22]:
scoring_based_capped = make_capped_renderer(scoring_based_prompt, CAPS)

In [23]:
import random

ASPECTS = ["challenge", "approach", "outcome"]

def scoring_based_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42):
    assert target_label in ASPECTS, f"target_label must be one of {ASPECTS}"

    rng = random.Random(seed)

    # --- choose one validation example that has at least one positive for this aspect ---
    idxs = list(range(len(val_dataset)))
    rng.shuffle(idxs)
    chosen = None
    for i in idxs:
        ex = val_dataset[i]
        labs = ex[f"{target_label}_labels"]
        if any(int(v) == 1 for v in labs):
            chosen = ex
            break
    if chosen is None:
        chosen = val_dataset[0]

    ex_sents = chosen["source_sentences"]
    ex_labs  = [int(v) for v in chosen[f"{target_label}_labels"]]
    ex_gold  = [j+1 for j, v in enumerate(ex_labs) if v == 1]

    # --- fake scoring scheme: high score (4–5) if gold label=1, else low (1–2) ---
    example_scores = [5 if v == 1 else 2 for v in ex_labs]

    numbered_example = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
    numbered_input   = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example (one-shot from validation set; fixed seed):
Input:
{"\n".join(numbered_example)}

Scoring process (illustrative, not to be returned):
For the target aspect "{target_label}", assign a score 1–5 to each sentence for aspect relevance:
{"; ".join([f"Sentence {i+1} → {sc}" for i, sc in enumerate(example_scores)])}

Select sentences with score ≥ 4:
Selected indices: {ex_gold}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1. For each sentence, assign a score from 1 (low relevance) to 5 (high relevance) for how well it expresses the "{target_label}" aspect.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON. Do not include explanations or the scores in your final output.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [24]:
scoring_based_one_shot_capped = make_capped_renderer(scoring_based_one_shot, CAPS)

**self ask**: The model generates internal questions like “What is this document about?” and selects the sentences that answer them.

In [25]:
def self_ask_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. Reason step-by-step. For each sentence, ask the question: "Does this sentence primarily express the "{target_label}" aspect?"
2. Answer the question with "Yes" or "No".
3. Compile a list of all sentences for which the answer was "Yes".
4. If no sentences meet the criteria, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [26]:
self_ask_capped = make_capped_renderer(self_ask_prompt, CAPS)

In [27]:
import random

ASPECTS = ["challenge", "approach", "outcome"]

def self_ask_one_shot(sentences, target_label, val_dataset=val_dataset, seed=42):
    assert target_label in ASPECTS, f"target_label must be one of {ASPECTS}"

    rng = random.Random(seed)

    # --- pick a validation example that has at least one positive for this aspect ---
    idxs = list(range(len(val_dataset)))
    rng.shuffle(idxs)
    chosen = None
    for i in idxs:
        ex = val_dataset[i]
        labs = ex[f"{target_label}_labels"]
        if any(int(v) == 1 for v in labs):
            chosen = ex
            break
    if chosen is None:
        chosen = val_dataset[0]

    ex_sents = chosen["source_sentences"]
    ex_labs  = [int(v) for v in chosen[f"{target_label}_labels"]]
    ex_gold  = [j+1 for j, v in enumerate(ex_labs) if v == 1]

    # --- construct Yes/No decisions for the example ---
    example_decisions = ["Yes" if v == 1 else "No" for v in ex_labs]

    numbered_example = [f"Sentence {i+1}: {s}" for i, s in enumerate(ex_sents)]
    numbered_input   = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Example (one-shot from validation set; fixed seed):
Input:
{"\n".join(numbered_example)}

Decision log (concise, not to be returned):
For the target aspect "{target_label}", answer Yes/No for each sentence:
{"; ".join([f"Sentence {i+1} → {d}" for i, d in enumerate(example_decisions)])}

Selected indices (Yes answers): {ex_gold}

---

New task:
Input:
{"\n".join(numbered_input)}

Instructions:
1. For each sentence, ask: "Does this sentence primarily express the '{target_label}' aspect?"
2. Answer each with "Yes" or "No" (internally).
3. Select all sentences answered "Yes".
4. If none, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON. Do not include the questions, answers, or any explanations in the final output.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


In [28]:
self_ask_one_shot_capped = make_capped_renderer(self_ask_one_shot, CAPS)

- Transform gold standard to match output for each label:

  so for example if the challenge_labels looks like this:

  `[1,0,0,1,1,0]`

  it will become

  `[1, 4, 5]`

- helper functions

In [29]:
import json, re

def safe_extract_json_strict(text: str):
    text = text.strip()

    try:
        js = json.loads(text)
        if isinstance(js, dict) and "selected_sentences" in js:
            return js
        if isinstance(js, list):
            return {"selected_sentences": js}
    except Exception:
        pass

    text_clean = re.sub(r"^```[\w-]*\s*\n", "", text, flags=re.S)
    text_clean = re.sub(r"\n```$", "", text_clean, flags=re.S).strip()

    objs = re.findall(r"\{[\s\S]*?\}", text_clean)
    for s in reversed(objs):
        try:
            js = json.loads(s)
            if isinstance(js, dict) and "selected_sentences" in js:
                return js
        except Exception:
            continue

    m = re.search(r"(?m)^\s*\[(?:\s*\d+\s*(?:,\s*\d+\s*)*)?\]\s*$", text_clean)
    if m:
        try:
            arr = json.loads(m.group(0))
            return {"selected_sentences": arr}
        except Exception:
            pass

    return {}


- for rouge calculation the abstractive dataset is needed

In [30]:
from datasets import load_dataset
from rouge_score import rouge_scorer
import numpy as np
import pandas as pd

# Load ACLSum splits
extractive_test = dataset
abstractive_test = load_dataset("sobamchan/aclsum", "abstractive")["test"]

# Map: doc id -> {"challenge": str, "approach": str, "outcome": str}
ABS_ASPECTS = ["challenge", "approach", "outcome"]
abs_by_id = {
    ex["id"]: {a: ex[a] for a in ABS_ASPECTS}
    for ex in abstractive_test
}

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

- prediction functions for the two different models

- The qwen model is pretrained on chat-like inputs, and therefore the input we have needs to be translated as a chat as well, and needs a system role

- llama does not use this in the same way as Qwen and the system instruction is typically baked into the user prompt or handled differently

These functions are kept in order to understand how the predictions were performed before, but this lead to the runtime being way too high to perform the analysis, even with the paid Colab GPU.

In previous run, these functions yielded predictions which were a bit better.

In [31]:
def _chunks(lst, size):
    for i in range(0, len(lst), size):
        yield i, lst[i:i+size]

In [32]:
def predict_indices_for_aspect_qwen(
    model,
    tokenizer,
    prompt_technique,
    sentences,
    target_label,
    max_new_tokens=128,
    show_raw=False,
    window=10,  # this should make things faster
    cap = False
):
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token

    all_idxs = []
    sent_iter = _chunks(sentences, window) if window else [(0, sentences)]

    for start, sub in sent_iter:
        # 1) Render prompt for this chunk
        user_prompt = prompt_technique(sub, target_label)

        # 2) Chat-format for Qwen
        chat_text = tokenizer.apply_chat_template(
            [
                {"role": "system", "content": "You are an expert in extractive summarization."},
                {"role": "user",   "content": user_prompt},
            ],
            tokenize=False,
            add_generation_prompt=True,
        )

        # 3) Tokenize
        inputs = tokenizer(chat_text, return_tensors="pt", padding=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # (optional) see prompt length
        # print(f"length={inputs['input_ids'].shape[1]}")

        # 4) Generate deterministically
        with torch.inference_mode():
            gen = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        # 5) Decode ONLY the newly generated tokens (exclude prompt)
        prompt_len = inputs["input_ids"].shape[1]
        gen_ids = gen[0, prompt_len:]                 # tail only
        out = tokenizer.decode(gen_ids, skip_special_tokens=True)

        if show_raw:
            print(f"\n[RAW OUTPUT {target_label} | chunk@{start}]\n{out}\n")

        # 6) Strict JSON parse → indices; shift by chunk start
        js = safe_extract_json_strict(out)
        idxs = js.get("selected_sentences", [])

        n = len(sub)
        for v in idxs:
            if isinstance(v, (int, float)):
                v = int(v)
                if 1 <= v <= n:
                    all_idxs.append(v + start)  # 1-based shift

        pred = sorted(set(all_idxs))

        if cap==True:
          # ENFORCE CAP:
          capK = CAPS.get(target_label)
          pred = _cap_indices_in_order(pred, capK)

    return pred

In [33]:
def predict_indices_for_aspect_llama(
    model,
    tokenizer,
    prompt_technique,
    sentences,
    target_label,
    max_new_tokens=128,     # tighter default
    show_raw=False,
    window=10,  # this should make things faster
    cap=False
):
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token

    all_idxs = []

    sent_iter = _chunks(sentences, window) if window else [(0, sentences)]
    for start, sub in sent_iter:
        # 1) Render prompt for this chunk
        user_prompt = prompt_technique(sub, target_label)

        # 2) Chat-format for LLaMA instruct (user-only is fine)
        chat_text = tokenizer.apply_chat_template(
            [{"role": "user", "content": user_prompt}],
            tokenize=False,
            add_generation_prompt=True,
        )

        # 3) Tokenize → generate deterministically (inference mode)
        inputs = tokenizer(chat_text, return_tensors="pt", padding=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        gen = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

        # decode only the generated tail
        prompt_len = inputs["input_ids"].shape[1]
        gen_ids = gen[0, prompt_len:]          # <-- new tokens only
        out = tokenizer.decode(gen_ids, skip_special_tokens=True)


        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        out = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if show_raw:
            print(f"\n[RAW OUTPUT {target_label} | chunk@{start}] {out}\n")

        # 4) Parse JSON → indices (strict) and shift to global 1-based
        js = safe_extract_json_strict(out)
        idxs = js.get("selected_sentences", [])

        n = len(sub)
        for v in idxs:
            if isinstance(v, (int, float)):
                v = int(v)
                if 1 <= v <= n:
                    all_idxs.append(v + start)   # shift by chunk start

        pred = sorted(set(all_idxs))

        if cap==True:
          # ENFORCE CAP:
          capK = CAPS.get(target_label)
          pred = _cap_indices_in_order(pred, capK)

    return pred

- metric calculation

As requested, the following metrics are calculated to evaluate model's performance:
- precision, recall and F1: they are the most straightforward metrics and they represent in order, the ratio of TP/(TP+FP), TP/(TP+FN) and the harmonic mean of precision and recall. They offer the principal analysis against the gold standard to understand how well the two models perform the predictions.
- ROUGE-1: unigram (word-level) overlap between the predicted summary and the human abstractive summary
- ROUGE-2: bigram overlap, more sensitive to fluency and local coherence.
- ROUGE-L: longest common subsequence, capturing sentence-level structural similarity.

Here, ROUGE provides a secondary perspective: while precision/recall/F1 check if the exact gold extractive sentences were found, ROUGE checks how much the extracted sentences overlap with the human abstractive references. This connects extractive performance to human-oriented summaries.

- Oracle ROUGE (per aspect and union): For each document, we construct an oracle summary by greedily selecting the subset of sentences that maximizes ROUGE with respect to the human abstractive summary. This represents the upper bound of predictions. The Oracle gap (RL) is the difference between model ROUGE-L and oracle ROUGE-L. A large gap means the model is far from the extractive upper bound, while a small gap means the model is nearly optimal under extractive constraints.

In [34]:
# Oracle (greedy ROUGE-L F1 upper bound)
_oracle_rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

def build_oracle_indices(sentences, abstr_ref, max_len=None):
    selected, remaining, best_f = [], list(range(len(sentences))), 0.0
    while remaining and (max_len is None or len(selected) < max_len):
        best_pair = (best_f, None)
        for i in remaining:
            hyp = " ".join([sentences[j] for j in selected + [i]])
            f = _oracle_rouge.score(abstr_ref, hyp)["rougeL"].fmeasure
            if f > best_pair[0]:
                best_pair = (f, i)
        if best_pair[1] is None:
            break
        best_f, chosen = best_pair
        selected.append(chosen)
        remaining.remove(chosen)
    return [i+1 for i in selected]  # 1-based

# Exact-match metrics on indices
def calculate_metrics(gold_indices, pred_indices):
    gs, ps = set(gold_indices), set(pred_indices)
    tp = len(gs & ps); fp = len(ps - gs); fn = len(gs - ps)
    p = tp/(tp+fp) if (tp+fp) else 0.0
    r = tp/(tp+fn) if (tp+fn) else 0.0
    f = 2*p*r/(p+r) if (p+r) else 0.0
    return {"precision": p, "recall": r, "f1_score": f, "tp": tp, "fp": fp, "fn": fn}

# for union calculation
def _concat_abs_refs(abs_ref):
    parts = []
    for k in ("challenge", "approach", "outcome"):
        if abs_ref and abs_ref.get(k):
            parts.append(abs_ref[k])
    return " ".join(parts)

- evaluation loop

This is the core of the project. The function takes as input the dataset, model (model name and tokenizer) as well as the prompt technique that needs to be used, the truth label for the whole dataset, the aspects (challenge, approach, outcome) and the truth labels taken from the abstractive dataset for the calculation of rouge scores. The following inputs are a series of flags which indicate how many documents to analyze, regulates the prints inside the function, the max length of the oracle and finally if the capping explained before is applied or not to enforce a max length of elements in the output array.

The goal of the function is to call the correct prediction method (with either Llama or Qwen) and compute the metrics which underline the quality of the results.

The problem is tackled in two ways:

- *aspect-based*: compute metrics according to each aspect, to understand if there are some which are more challenging compared to others.

- *aspect-agnostic*: compute the metrics according to all aspects together to understand the general prediction power of the model. This is done through union (which aggregates the predicted values and gold standard before calculating the metrics on that), and overall, which instead takes the final values of all aspects (+union) and calculates the mean to have a good understanding of model performance.

Both these aspects are necessary to get a comprehensive view of performance.

In [35]:
# def run_evaluation_with_rouge_and_oracle(
#     dataset,
#     model_name,
#     model,
#     tokenizer,
#     prompt_technique,
#     gold_all,
#     aspects,
#     abs_by_id,
#     start=0,
#     stop=None,
#     verbose=False,
#     oracle_max_len=None,
#     capped=False,
# ):
#     if stop is None:
#         stop = len(dataset)
#     stop = min(stop, len(dataset))

#     # Aggregators
#     agg = {a: {"P": [], "R": [], "F1": []} for a in aspects}
#     rouge_sc = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
#     r_model = {a: {"r1": [], "r2": [], "rL": []} for a in aspects}
#     r_oracl = {a: {"r1": [], "r2": [], "rL": []} for a in aspects}
#     r_gap   = {a: {"rL_gap": []} for a in aspects}
#     per_doc_rows = []

#     # We’ll append "union" later for summaries
#     union_key = "union"
#     union_initialized = False

#     for doc_idx in range(start, stop):
#         ex = dataset[doc_idx]
#         doc_id    = ex["id"]
#         sentences = ex["source_sentences"]
#         gold_std  = gold_all[doc_idx]
#         abs_ref   = abs_by_id.get(doc_id, None)

#         if verbose:
#             print("="*80)
#             print(f"Doc {doc_idx} (id={doc_id}) | #sentences={len(sentences)}")
#             print("-"*80)

#         # keep per-aspect predictions to compute union
#         preds_this_doc = {}

#         # ---------- PER-ASPECT ----------
#         for aspect in aspects:
#             if "Qwen" in model_name:
#               if capped == False:
#                   pred = predict_indices_for_aspect_qwen(
#                       model=model, tokenizer=tokenizer,
#                       prompt_technique=prompt_technique,
#                       sentences=sentences, target_label=aspect, cap = False
#                   )
#               else:
#                   pred = predict_indices_for_aspect_qwen(
#                       model=model, tokenizer=tokenizer,
#                       prompt_technique=prompt_technique,
#                       sentences=sentences, target_label=aspect, cap = True
#                   )
#             elif "Llama" in model_name or "LLaMA" in model_name:
#               if capped == False:
#                   pred = predict_indices_for_aspect_llama(
#                       model=model, tokenizer=tokenizer,
#                       prompt_technique=prompt_technique,
#                       sentences=sentences, target_label=aspect, cap = False
#                   )
#               else:
#                 pred = predict_indices_for_aspect_llama(
#                     model=model, tokenizer=tokenizer,
#                     prompt_technique=prompt_technique,
#                     sentences=sentences, target_label=aspect, cap=True
#                 )
#             else:
#                 pred = []

#             preds_this_doc[aspect] = pred
#             gold_idx = gold_std.get(aspect, [])

#             # exact-match
#             m = calculate_metrics(gold_idx, pred)
#             agg[aspect]["P"].append(m["precision"])
#             agg[aspect]["R"].append(m["recall"])
#             agg[aspect]["F1"].append(m["f1_score"])

#             # ROUGE vs abstractive (model + oracle)
#             r1=r2=rL=0.0
#             or_r1=or_r2=or_rL=0.0
#             gap=0.0
#             oracle_idx=[]

#             if abs_ref and abs_ref.get(aspect):
#                 ref = abs_ref[aspect]
#                 hyp_model = indices_to_text(sentences, pred)
#                 rs_m = rouge_sc.score(ref, hyp_model)
#                 r1, r2, rL = rs_m["rouge1"].fmeasure, rs_m["rouge2"].fmeasure, rs_m["rougeL"].fmeasure
#                 r_model[aspect]["r1"].append(r1); r_model[aspect]["r2"].append(r2); r_model[aspect]["rL"].append(rL)

#                 oracle_idx = build_oracle_indices(sentences, ref, max_len=oracle_max_len)
#                 hyp_or = indices_to_text(sentences, oracle_idx)
#                 rs_o = rouge_sc.score(ref, hyp_or)
#                 or_r1, or_r2, or_rL = rs_o["rouge1"].fmeasure, rs_o["rouge2"].fmeasure, rs_o["rougeL"].fmeasure
#                 r_oracl[aspect]["r1"].append(or_r1); r_oracl[aspect]["r2"].append(or_r2); r_oracl[aspect]["rL"].append(or_rL)
#                 gap = or_rL - rL
#                 r_gap[aspect]["rL_gap"].append(gap)

#             if verbose:
#                 print(f"  -> {aspect.capitalize():9s}")
#                 print(f"     Predicted: {sorted(pred)}")
#                 print(f"     Gold:      {sorted(gold_idx)}")
#                 print(f"     Metrics:   P={m['precision']:.2f} R={m['recall']:.2f} F1={m['f1_score']:.2f}")
#                 if abs_ref and abs_ref.get(aspect):
#                     print(f"     ROUGE F1 (model):  R1={r1:.3f}  R2={r2:.3f}  RL={rL:.3f}")
#                     print(f"     ROUGE F1 (oracle): R1={or_r1:.3f} R2={or_r2:.3f} RL={or_rL:.3f}")
#                     print(f"     Oracle gap (RL):   {gap:+.3f}")

#             per_doc_rows.append({
#                 "technique": prompt_technique,
#                 "model_name": model_name,
#                 "doc_idx": doc_idx,
#                 "doc_id": doc_id,
#                 "aspect": aspect,
#                 "num_sentences": len(sentences),
#                 "pred_indices": sorted(pred),
#                 "gold_indices": sorted(gold_idx),
#                 "precision": m["precision"],
#                 "recall": m["recall"],
#                 "f1": m["f1_score"],
#                 "rouge1_model": r1, "rouge2_model": r2, "rougeL_model": rL,
#                 "rouge1_oracle": or_r1, "rouge2_oracle": or_r2, "rougeL_oracle": or_rL,
#                 "oracle_gap_rougeL": gap,
#                 "oracle_indices": oracle_idx,
#             })

#         # ---------- UNION (aspect-agnostic) ----------
#         gold_union = sorted(set().union(*[set(gold_std.get(a, [])) for a in aspects]))
#         pred_union = sorted(set().union(*[set(preds_this_doc.get(a, [])) for a in aspects]))
#         mU = calculate_metrics(gold_union, pred_union)

#         if not union_initialized:
#             agg[union_key] = {"P": [], "R": [], "F1": []}
#             r_model[union_key] = {"r1": [], "r2": [], "rL": []}
#             r_oracl[union_key] = {"r1": [], "r2": [], "rL": []}
#             r_gap[union_key]   = {"rL_gap": []}
#             union_initialized = True

#         agg[union_key]["P"].append(mU["precision"])
#         agg[union_key]["R"].append(mU["recall"])
#         agg[union_key]["F1"].append(mU["f1_score"])

#         r1U=r2U=rLU=0.0
#         or_r1U=or_r2U=or_rLU=0.0
#         gapU=0.0
#         oracle_union_idx=[]

#         if abs_ref:
#             refU = _concat_abs_refs(abs_ref)
#             hypU = indices_to_text(sentences, pred_union)
#             rsU  = rouge_sc.score(refU, hypU)
#             r1U, r2U, rLU = rsU["rouge1"].fmeasure, rsU["rouge2"].fmeasure, rsU["rougeL"].fmeasure
#             r_model[union_key]["r1"].append(r1U); r_model[union_key]["r2"].append(r2U); r_model[union_key]["rL"].append(rLU)

#             oracle_union_idx = build_oracle_indices(sentences, refU, max_len=oracle_max_len)
#             hyp_orU = indices_to_text(sentences, oracle_union_idx)
#             rs_orU  = rouge_sc.score(refU, hyp_orU)
#             or_r1U, or_r2U, or_rLU = rs_orU["rouge1"].fmeasure, rs_orU["rouge2"].fmeasure, rs_orU["rougeL"].fmeasure
#             r_oracl[union_key]["r1"].append(or_r1U); r_oracl[union_key]["r2"].append(or_r2U); r_oracl[union_key]["rL"].append(or_rLU)
#             gapU = or_rLU - rLU
#             r_gap[union_key]["rL_gap"].append(gapU)

#         if verbose:
#             print(f"  -> {'Union':9s}")
#             print(f"     Predicted: {pred_union}")
#             print(f"     Gold:      {gold_union}")
#             print(f"     Metrics:   P={mU['precision']:.2f} R={mU['recall']:.2f} F1={mU['f1_score']:.2f}")
#             if abs_ref:
#                 print(f"     ROUGE F1 (model):  R1={r1U:.3f}  R2={r2U:.3f}  RL={rLU:.3f}")
#                 print(f"     ROUGE F1 (oracle): R1={or_r1U:.3f} R2={or_r2U:.3f} RL={or_rLU:.3f}")
#                 print(f"     Oracle gap (RL):   {gapU:+.3f}")

#         per_doc_rows.append({
#             "technique": prompt_technique,
#             "model_name": model_name,
#             "doc_idx": doc_idx,
#             "doc_id": doc_id,
#             "aspect": "union",
#             "num_sentences": len(sentences),
#             "pred_indices": pred_union,
#             "gold_indices": gold_union,
#             "precision": mU["precision"],
#             "recall": mU["recall"],
#             "f1": mU["f1_score"],
#             "rouge1_model": r1U, "rouge2_model": r2U, "rougeL_model": rLU,
#             "rouge1_oracle": or_r1U, "rouge2_oracle": or_r2U, "rougeL_oracle": or_rLU,
#             "oracle_gap_rougeL": gapU,
#             "oracle_indices": oracle_union_idx,
#         })

#         if verbose:
#             print()

#     # --------- Aggregate summaries (per aspect + union + overall) ---------
#     all_aspects = aspects + (["union"] if union_initialized else [])

#     per_aspect_summary = {}
#     for a in all_aspects:
#         P,R,F = agg[a]["P"], agg[a]["R"], agg[a]["F1"]
#         r1m,r2m,rLm = r_model[a]["r1"], r_model[a]["r2"], r_model[a]["rL"]
#         r1o,r2o,rLo = r_oracl[a]["r1"], r_oracl[a]["r2"], r_oracl[a]["rL"]
#         gaps = r_gap[a]["rL_gap"]
#         per_aspect_summary[a] = {
#             "precision": float(np.mean(P)) if P else 0.0,
#             "recall":    float(np.mean(R)) if R else 0.0,
#             "f1":        float(np.mean(F)) if F else 0.0,
#             "rouge1_model": float(np.mean(r1m)) if r1m else 0.0,
#             "rouge2_model": float(np.mean(r2m)) if r2m else 0.0,
#             "rougeL_model": float(np.mean(rLm)) if rLm else 0.0,
#             "rouge1_oracle": float(np.mean(r1o)) if r1o else 0.0,
#             "rouge2_oracle": float(np.mean(r2o)) if r2o else 0.0,
#             "rougeL_oracle": float(np.mean(rLo)) if rLo else 0.0,
#             "oracle_gap_rougeL": float(np.mean(gaps)) if gaps else 0.0,
#         }

#     overall_macro = {
#         k: float(np.mean([per_aspect_summary[a][k] for a in all_aspects])) if all_aspects else 0.0
#         for k in ["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
#                   "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"]
#     }

#     return {
#         "technique": prompt_technique,     # use a string name, not the function object
#         "model_name": model_name,
#         "per_aspect": per_aspect_summary,  # includes "union"
#         "overall_macro": overall_macro,
#         "per_doc": per_doc_rows,
#     }


In [47]:
from rouge_score import rouge_scorer
import numpy as np
import torch

# build one batched prompt per aspect
def _build_aspect_batch(tokenizer, prompt_fn, sentences, aspects, system_msg=None):
    prompts = []
    for a in aspects:
        user = prompt_fn(sentences, a)
        messages = ([{"role": "system", "content": system_msg}] if system_msg else []) + [
            {"role": "user", "content": user}
        ]
        chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        prompts.append(chat)
    toks = tokenizer(prompts, return_tensors="pt", padding=True)
    prompt_lens = toks["attention_mask"].sum(dim=1)  # tensor per sample
    return toks, prompt_lens

def run_evaluation_with_rouge_and_oracle(
    dataset,
    model_name,
    model,
    tokenizer,
    prompt_technique,
    gold_all,
    aspects,
    abs_by_id,
    start=0,
    stop=None,
    verbose=False,
    oracle_max_len=None,
    capped=False,
):
    """
    Requirements (already in your notebook):
      - calculate_metrics(gold_indices, pred_indices) -> dict with precision/recall/f1
      - indices_to_text(sentences, idxs_1b) -> str
      - build_oracle_indices(sentences, abstr_ref, max_len=None) -> [1-based idx]
      - If you use `capped=True`, define CAPS (dict) and _cap_indices_in_order(indices, cap)

    Returns dict with:
      - technique, model_name
      - per_aspect (includes 'union')
      - overall_macro
      - per_doc rows (ready for DataFrame)
    """
    if stop is None:
        stop = len(dataset)
    stop = min(stop, len(dataset))

    # concat three abstractive refs for union
    def _concat_abs_refs(abs_ref):
        parts = []
        for k in ("challenge", "approach", "outcome"):
            if abs_ref and abs_ref.get(k):
                parts.append(abs_ref[k])
        return " ".join(parts)

    # aggregators
    agg = {a: {"P": [], "R": [], "F1": []} for a in aspects}
    rouge_sc = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    r_model = {a: {"r1": [], "r2": [], "rL": []} for a in aspects}
    r_oracl = {a: {"r1": [], "r2": [], "rL": []} for a in aspects}
    r_gap   = {a: {"rL_gap": []} for a in aspects}
    per_doc_rows = []

    union_key = "union"
    union_initialized = False

    # short, greedy outputs are enough (JSON)
    GEN_KW = dict(max_new_tokens=64, do_sample=False, use_cache=True)

    for doc_idx in range(start, stop):
        ex = dataset[doc_idx]
        doc_id    = ex["id"]
        sentences = ex["source_sentences"]
        gold_std  = gold_all[doc_idx]
        abs_ref   = abs_by_id.get(doc_id, None)

        if verbose:
            print("="*80)
            print(f"Doc {doc_idx} (id={doc_id}) | #sentences={len(sentences)}")
            print("-"*80)

        # ---- PER-ASPECT: do ONE generate() for all aspects of this doc
        is_qwen = ("Qwen" in model_name)
        system_msg = "You are an expert in extractive summarization." if is_qwen else None

        toks, prompt_lens = _build_aspect_batch(
            tokenizer=tokenizer,
            prompt_fn=prompt_technique,
            sentences=sentences,
            aspects=aspects,
            system_msg=system_msg,
        )
        toks = {k: v.to(model.device, non_blocking=True) for k, v in toks.items()}

        with torch.inference_mode():
            out_ids = model.generate(
                **toks,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                **GEN_KW
            )

        # decode tails & parse JSON -> indices per aspect
        preds_this_doc = {}
        for a, row, plen in zip(aspects, out_ids, prompt_lens):
            tail_ids = row[int(plen):]
            out_str = tokenizer.decode(tail_ids, skip_special_tokens=True)

            js = safe_extract_json_strict(out_str)  # your strict parser
            idxs = js.get("selected_sentences", [])

            # sanitize 1-based indices
            n = len(sentences)
            clean = []
            for v in idxs:
                if isinstance(v, (int, float)):
                    v = int(v)
                    if 1 <= v <= n:
                        clean.append(v)
            clean = sorted(set(clean))

            # optional cap
            if capped and "CAPS" in globals():
                capK = CAPS.get(a, None)
                if capK is not None and "_cap_indices_in_order" in globals():
                    clean = _cap_indices_in_order(clean, capK)

            preds_this_doc[a] = clean

        # ---- metrics/rouge/oracle per aspect
        for aspect in aspects:
            pred = preds_this_doc[aspect]
            gold_idx = gold_std.get(aspect, [])

            # exact-match metrics
            m = calculate_metrics(gold_idx, pred)
            agg[aspect]["P"].append(m["precision"])
            agg[aspect]["R"].append(m["recall"])
            agg[aspect]["F1"].append(m["f1_score"])

            # ROUGE vs abstractive + oracle
            r1=r2=rL=0.0
            or_r1=or_r2=or_rL=0.0
            gap=0.0
            oracle_idx=[]

            if abs_ref and abs_ref.get(aspect):
                ref = abs_ref[aspect]
                hyp_model = indices_to_text(sentences, pred)

                rs_m = rouge_sc.score(ref, hyp_model)
                r1, r2, rL = rs_m["rouge1"].fmeasure, rs_m["rouge2"].fmeasure, rs_m["rougeL"].fmeasure
                r_model[aspect]["r1"].append(r1)
                r_model[aspect]["r2"].append(r2)
                r_model[aspect]["rL"].append(rL)

                oracle_idx = build_oracle_indices(sentences, ref, max_len=oracle_max_len)
                hyp_or = indices_to_text(sentences, oracle_idx)
                rs_o = rouge_sc.score(ref, hyp_or)
                or_r1, or_r2, or_rL = rs_o["rouge1"].fmeasure, rs_o["rouge2"].fmeasure, rs_o["rougeL"].fmeasure
                r_oracl[aspect]["r1"].append(or_r1)
                r_oracl[aspect]["r2"].append(or_r2)
                r_oracl[aspect]["rL"].append(or_rL)
                gap = or_rL - rL
                r_gap[aspect]["rL_gap"].append(gap)

            if verbose:
                print(f"  -> {aspect.capitalize():9s}")
                print(f"     Predicted: {pred}")
                print(f"     Gold:      {sorted(gold_idx)}")
                print(f"     Metrics:   P={m['precision']:.2f} R={m['recall']:.2f} F1={m['f1_score']:.2f}")
                if abs_ref and abs_ref.get(aspect):
                    print(f"     ROUGE F1 (model):  R1={r1:.3f}  R2={r2:.3f}  RL={rL:.3f}")
                    print(f"     ROUGE F1 (oracle): R1={or_r1:.3f} R2={or_r2:.3f} RL={or_rL:.3f}")
                    print(f"     Oracle gap (RL):   {gap:+.3f}")

            per_doc_rows.append({
                "technique": prompt_technique,  # you overwrite with readable name in caller
                "model_name": model_name,
                "doc_idx": doc_idx,
                "doc_id": doc_id,
                "aspect": aspect,
                "num_sentences": len(sentences),
                "pred_indices": pred,
                "gold_indices": sorted(gold_idx),
                "precision": m["precision"],
                "recall": m["recall"],
                "f1": m["f1_score"],
                "rouge1_model": r1, "rouge2_model": r2, "rougeL_model": rL,
                "rouge1_oracle": or_r1, "rouge2_oracle": or_r2, "rougeL_oracle": or_rL,
                "oracle_gap_rougeL": gap,
                "oracle_indices": oracle_idx,
            })

        # ---- UNION (aspect-agnostic)
        gold_union = sorted(set().union(*[set(gold_std.get(a, [])) for a in aspects]))
        pred_union = sorted(set().union(*[set(preds_this_doc.get(a, [])) for a in aspects]))
        mU = calculate_metrics(gold_union, pred_union)

        if not union_initialized:
            agg[union_key] = {"P": [], "R": [], "F1": []}
            r_model[union_key] = {"r1": [], "r2": [], "rL": []}
            r_oracl[union_key] = {"r1": [], "r2": [], "rL": []}
            r_gap[union_key]   = {"rL_gap": []}
            union_initialized = True

        agg[union_key]["P"].append(mU["precision"])
        agg[union_key]["R"].append(mU["recall"])
        agg[union_key]["F1"].append(mU["f1_score"])

        r1U=r2U=rLU=0.0
        or_r1U=or_r2U=or_rLU=0.0
        gapU=0.0
        oracle_union_idx=[]

        if abs_ref:
            refU = _concat_abs_refs(abs_ref)
            hypU = indices_to_text(sentences, pred_union)
            rsU  = rouge_sc.score(refU, hypU)
            r1U, r2U, rLU = rsU["rouge1"].fmeasure, rsU["rouge2"].fmeasure, rsU["rougeL"].fmeasure
            r_model[union_key]["r1"].append(r1U)
            r_model[union_key]["r2"].append(r2U)
            r_model[union_key]["rL"].append(rLU)

            oracle_union_idx = build_oracle_indices(sentences, refU, max_len=oracle_max_len)
            hyp_orU = indices_to_text(sentences, oracle_union_idx)
            rs_orU  = rouge_sc.score(refU, hyp_orU)
            or_r1U, or_r2U, or_rLU = rs_orU["rouge1"].fmeasure, rs_orU["rouge2"].fmeasure, rs_orU["rougeL"].fmeasure
            r_oracl[union_key]["r1"].append(or_r1U)
            r_oracl[union_key]["r2"].append(or_r2U)
            r_oracl[union_key]["rL"].append(or_rLU)
            gapU = or_rLU - rLU
            r_gap[union_key]["rL_gap"].append(gapU)

        if verbose:
            print(f"  -> {'Union':9s}")
            print(f"     Predicted: {pred_union}")
            print(f"     Gold:      {gold_union}")
            print(f"     Metrics:   P={mU['precision']:.2f} R={mU['recall']:.2f} F1={mU['f1_score']:.2f}")
            if abs_ref:
                print(f"     ROUGE F1 (model):  R1={r1U:.3f}  R2={r2U:.3f}  RL={rLU:.3f}")
                print(f"     ROUGE F1 (oracle): R1={or_r1U:.3f} R2={or_r2U:.3f} RL={or_rLU:.3f}")
                print(f"     Oracle gap (RL):   {gapU:+.3f}")

        per_doc_rows.append({
            "technique": prompt_technique,
            "model_name": model_name,
            "doc_idx": doc_idx,
            "doc_id": doc_id,
            "aspect": "union",
            "num_sentences": len(sentences),
            "pred_indices": pred_union,
            "gold_indices": gold_union,
            "precision": mU["precision"],
            "recall": mU["recall"],
            "f1": mU["f1_score"],
            "rouge1_model": r1U, "rouge2_model": r2U, "rougeL_model": rLU,
            "rouge1_oracle": or_r1U, "rouge2_oracle": or_r2U, "rougeL_oracle": or_rLU,
            "oracle_gap_rougeL": gapU,
            "oracle_indices": oracle_union_idx,
        })

        if verbose:
            print()

    # ---- aggregate (per aspect + union + overall)
    all_aspects = aspects + (["union"] if union_initialized else [])

    per_aspect_summary = {}
    for a in all_aspects:
        P,R,F = agg[a]["P"], agg[a]["R"], agg[a]["F1"]
        r1m,r2m,rLm = r_model[a]["r1"], r_model[a]["r2"], r_model[a]["rL"]
        r1o,r2o,rLo = r_oracl[a]["r1"], r_oracl[a]["r2"], r_oracl[a]["rL"]
        gaps = r_gap[a]["rL_gap"]
        per_aspect_summary[a] = {
            "precision": float(np.mean(P)) if P else 0.0,
            "recall":    float(np.mean(R)) if R else 0.0,
            "f1":        float(np.mean(F)) if F else 0.0,
            "rouge1_model": float(np.mean(r1m)) if r1m else 0.0,
            "rouge2_model": float(np.mean(r2m)) if r2m else 0.0,
            "rougeL_model": float(np.mean(rLm)) if rLm else 0.0,
            "rouge1_oracle": float(np.mean(r1o)) if r1o else 0.0,
            "rouge2_oracle": float(np.mean(r2o)) if r2o else 0.0,
            "rougeL_oracle": float(np.mean(rLo)) if rLo else 0.0,
            "oracle_gap_rougeL": float(np.mean(gaps)) if gaps else 0.0,
        }

    overall_macro = {
        k: float(np.mean([per_aspect_summary[a][k] for a in all_aspects])) if all_aspects else 0.0
        for k in ["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                  "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"]
    }

    return {
        "technique": prompt_technique,   # overwrite with readable name in your outer loop
        "model_name": model_name,
        "per_aspect": per_aspect_summary,
        "overall_macro": overall_macro,
        "per_doc": per_doc_rows,
    }


- helper to transform the results into dataframes

In [48]:
def results_to_dataframe(results_list):
    rows_doc = []
    for res in results_list:
        rows_doc.extend(res["per_doc"])
    df_docs = pd.DataFrame(rows_doc)

    rows_sum = []
    for res in results_list:
        tech = res["technique"]; model = res["model_name"]
        row = {"technique": tech, "model_name": model}
        for a, vals in res["per_aspect"].items():
            for k, v in vals.items():
                row[f"{a}_{k}"] = v
        for k, v in res["overall_macro"].items():
            row[f"overall_{k}"] = v
        rows_sum.append(row)
    df_summary = pd.DataFrame(rows_sum)
    return df_docs, df_summary

- helper to clean said dataframes

In [63]:
import re
import pandas as pd
import numpy as np

def _tech_name(x):
    if callable(x):
        return x.__name__
    s = str(x)
    m = re.search(r"<function\s+([A-Za-z0-9_]+)\s+at\s+0x[0-9a-fA-F]+>", s)
    return m.group(1) if m else s

def make_three_aspect_tables(
    df_summary: pd.DataFrame,
    aspects=("challenge","approach","outcome", "union", "overall"),
    include_metrics=None,
    collapse_duplicates="mean",
    multi_model_columns=True,
    clean_tech_names=True,
):
    """
    Returns: (challenge_df, approach_df, outcome_df, union_df, overall_df)
    Each DF has:
      - rows = metrics for that aspect
      - columns = techniques  (or MultiIndex (model_name, technique) if multi_model_columns=True)
    Values are aggregated per technique (and model) if duplicates exist.
    """
    df = df_summary.copy()

    # clean technique column
    if "technique" in df.columns and clean_tech_names:
        df["technique"] = df["technique"].apply(_tech_name)

    # sanity: ensure model_name exists
    if "model_name" not in df.columns:
        df["model_name"] = "model"

    # helper to build one aspect table
    def _one_aspect_table(aspect):
        # pick all columns for this aspect
        asp_cols = [c for c in df.columns if c.startswith(f"{aspect}_")]
        if include_metrics:
            # restrict to requested metrics if provided
            wanted = {f"{aspect}_{m}" for m in include_metrics}
            asp_cols = [c for c in asp_cols if c in wanted]
        if not asp_cols:
            return pd.DataFrame()

        # melt to long: one row per (run, metric)
        long = df[["model_name","technique"] + asp_cols].melt(
            id_vars=["model_name","technique"],
            value_vars=asp_cols,
            var_name="metric_full",
            value_name="value",
        )
        # strip aspect_ prefix -> metric name only
        long["metric"] = long["metric_full"].str.replace(f"^{aspect}_", "", regex=True)
        long = long.drop(columns=["metric_full"])

        # (optional) collapse duplicates per (model, technique, metric)
        if collapse_duplicates:
            agg_fn = {"mean": np.mean, "max": np.max, "min": np.min}.get(collapse_duplicates, np.mean)
            long = (long
                    .groupby(["model_name","technique","metric"], as_index=False)["value"]
                    .aggregate(agg_fn))

        # pivot: rows=metric, cols=technique (or MultiIndex)
        if multi_model_columns and df["model_name"].nunique() > 1:
            wide = long.pivot_table(index="metric", columns=["model_name","technique"], values="value")
        else:
            wide = long.pivot_table(index="metric", columns="technique", values="value")

        # sort metrics nicely (precision, recall, f1 first)
        order_front = ["precision","recall","f1"]
        rest = [m for m in wide.index if m not in order_front]
        wide = wide.loc[[m for m in order_front if m in wide.index] + rest]

        # nice formatting
        wide = wide.sort_index(axis=1)
        return wide

    tables = []
    for a in aspects:
        tables.append(_one_aspect_table(a))

    # return in requested order: (challenge_df, approach_df, outcome_df)
    return tuple(tables)


- running the loops

In [49]:
techniques = [vanilla_prompt, least_to_most_prompt, least_to_most_one_shot,
              tool_augmented_prompt, tool_augmented_one_shot,
              self_ask_prompt, self_ask_one_shot,
              scoring_based_prompt, scoring_based_one_shot]

techniques_str = ["vanilla_prompt", "least_to_most_prompt", "least_to_most_one_shot",
              "tool_augmented_prompt", "tool_augmented_one_shot",
              "self_ask_prompt", "self_ask_one_shot",
              "scoring_based_prompt", "scoring_based_one_shot"]

techniques_capped = [vanilla_capped, least_to_most_capped, least_to_most_one_shot_capped, tool_augmented_capped, tool_augmented_one_shot_capped,
                     self_ask_capped, self_ask_one_shot_capped, scoring_based_capped, scoring_based_one_shot_capped]

techniques_capped_str = ["vanilla_capped", "least_to_most_capped", "least_to_most_one_shot_capped", "tool_augmented_capped", "tool_augmented_one_shot_capped",
                     "self_ask_capped", "self_ask_one_shot_capped", "scoring_based_capped", "scoring_based_one_shot_capped"]

- helper function to improve speed

In [50]:
def load_model_and_tokenizer(model_name: str):
    kwargs = dict(
        torch_dtype=torch.bfloat16,          # BF16 = faster & less memory
        device_map="auto",                   # automatically place on GPU
    )

    if "Qwen" in model_name:
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            **kwargs
        )
        print("Loaded Qwen model and tokenizer (with BF16).")

    elif "Llama" in model_name:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            **kwargs
        )
        print("Loaded Llama model and tokenizer (with BF16).")

    else:
        raise ValueError("Unsupported model name. Please use a Qwen or Llama model.")

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"  # good for batching

    return model.eval(), tokenizer


### Llama

In [51]:
model_name = "meta-llama/Llama-3.2-3B-Instruct" # Or "meta-llama/Llama-3.2-3B-Instruct" || Qwen/Qwen3-4B

model, tokenizer = load_model_and_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded Llama model and tokenizer (with BF16).


In [52]:
results = []
for tech in techniques:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = False
    )
    results.append(res)

df_docs, df_summary = results_to_dataframe(results)

Streaming output truncated to the last 5000 lines.
     Predicted: [2, 3, 5, 8, 9, 10, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.70 R=0.78 F1=0.74
     ROUGE F1 (model):  R1=0.281  R2=0.036  RL=0.180
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.063

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 16, 17, 23, 33]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.43 R=0.60 F1=0.50
     ROUGE F1 (model):  R1=0.144  R2=0.058  RL=0.077
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.389
  -> Approach 
     Predicted: []
     Gold:      [3, 19]
     Metrics:   P=0.00 R=0.00 F1=0.00
     ROUGE F1 (model):  R1=0.000  R2=0.000  RL=0.000
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.448
  -> Outcome  
     Predicted: [4, 25, 26, 27, 30, 33, 34, 35]


In [83]:
challenge_df, approach_df, outcome_df, union_df, overall_df = make_three_aspect_tables(
    df_summary,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)


/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x

There is a very interesting finding taken from this. When the prediction method is changed in favor of speed-oriented batching, the empirical behavior changes, even though the prompts are the same.

In [76]:
challenge_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.262193,0.131504,0.104671,0.128817,0.042201,0.085448,0.089949,0.063833,0.159508
recall,0.310500,0.235845,0.147083,0.265143,0.127429,0.160944,0.147381,0.101214,0.354615
f1,0.259903,0.157346,0.106003,0.161114,0.058432,0.104363,0.091633,0.073726,0.203356
oracle_gap_rougeL,0.258833,0.319242,0.301109,0.317218,0.330527,0.338287,0.307382,0.344370,0.302741
rouge1_model,0.160373,0.074690,0.103805,0.076492,0.060063,0.049379,0.094831,0.038103,0.098095
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.058235,0.029581,0.024340,0.029416,0.016816,0.017969,0.021013,0.014749,0.039873
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.112593,0.052184,0.070317,0.054208,0.040899,0.033139,0.064044,0.027056,0.068685


In [78]:
approach_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.303144,0.115511,0.282167,0.140318,0.208630,0.062036,0.254468,0.053306,0.167474
recall,0.329245,0.176507,0.280102,0.218766,0.227682,0.075258,0.339075,0.068897,0.300630
f1,0.256825,0.132011,0.246715,0.158286,0.186532,0.062419,0.265861,0.056107,0.205615
oracle_gap_rougeL,0.319371,0.416685,0.311755,0.405910,0.331195,0.429884,0.327066,0.432299,0.399893
rouge1_model,0.167999,0.038300,0.180051,0.052868,0.160380,0.019764,0.161074,0.017116,0.058669
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.067029,0.019734,0.072380,0.023576,0.052015,0.011209,0.067901,0.008359,0.030002
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.126807,0.029493,0.134424,0.040268,0.114984,0.016295,0.119113,0.013880,0.046286


In [79]:
outcome_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.325526,0.383223,0.279318,0.364817,0.324712,0.261159,0.319706,0.186897,0.363929
recall,0.384140,0.402595,0.166067,0.470365,0.306013,0.289841,0.338315,0.232127,0.430611
f1,0.318310,0.367014,0.195685,0.382573,0.285740,0.260400,0.302089,0.199763,0.365074
oracle_gap_rougeL,0.284213,0.312387,0.262861,0.320906,0.270906,0.356479,0.278770,0.370575,0.323069
rouge1_model,0.200919,0.151368,0.234216,0.143871,0.221714,0.097211,0.209877,0.075666,0.138743
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.077998,0.072747,0.066261,0.067372,0.078806,0.044721,0.076815,0.033814,0.069811
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.144259,0.116084,0.165610,0.107566,0.157565,0.071993,0.149701,0.057897,0.105403


To analyze briefly the results of this run, some things are noticeable:
- it looks like the vanilla prompt does not perform that badly, and methods like the tool augmented with zero shot seems to underperform consistently
- it also looks like the easiest aspect to detect is the outcome, while the hardest is the challenge.

**Why?**:
- Least-to-Most prompting achieved the best average performance, likely because the decomposition step clarifies aspect criteria and curbs over-selection
- Vanilla remained competitive: with clear aspect definitions, small open models exploit strong lexical markers (especially for Approach/Outcome) without the overhead of complex reasoning scaffolds
- for tool-augmented prompting, the extra scaffolding inflates the prompt length without adding real information. For a 3–4B model with limited context processing ability, this can crowd out useful signal from the actual sentences.
- for self-ask prompting techniques, small models tend to not remain consistent in the 30-40 sentence range that we have.
- Scoring-based, like tool-augmented, tends to waste tokens on meta-instructions instead of focusing on the sentence-aspect match.

In [84]:
union_df

technique,least_to_most_one_shot,least_to_most_prompt,scoring_based_one_shot,scoring_based_prompt,self_ask_one_shot,self_ask_prompt,tool_augmented_one_shot,tool_augmented_prompt,vanilla_prompt
metric,,,,,,,,,
precision,0.542272,0.475530,0.492826,0.477050,0.492249,0.320071,0.487526,0.252067,0.426276
recall,0.487089,0.302986,0.409564,0.368453,0.426365,0.197045,0.519366,0.152508,0.439574
f1,0.473014,0.333031,0.414789,0.380789,0.418985,0.224727,0.477503,0.176857,0.404464
oracle_gap_rougeL,0.189144,0.218913,0.189607,0.217687,0.191969,0.259938,0.205980,0.280346,0.226928
rouge1_model,0.280863,0.230191,0.282890,0.228870,0.276654,0.163861,0.250495,0.126761,0.206674
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.111570,0.085150,0.104423,0.088069,0.105904,0.058762,0.102346,0.046993,0.086188
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.166460,0.136691,0.165997,0.137916,0.163635,0.095666,0.149623,0.075257,0.128676


- capped results

In [54]:
results_capped = []
for tech in techniques_capped:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=True,
        oracle_max_len=None,
        capped = True
    )
    results_capped.append(res)

df_docs_capped, df_summary_capped = results_to_dataframe(results_capped)

Streaming output truncated to the last 5000 lines.
     Predicted: [2, 3, 5, 8, 9, 10, 12, 13, 14, 15]
     Gold:      [1, 2, 3, 7, 8, 9, 10, 13, 14]
     Metrics:   P=0.70 R=0.78 F1=0.74
     ROUGE F1 (model):  R1=0.281  R2=0.036  RL=0.180
     ROUGE F1 (oracle): R1=0.467 R2=0.038 RL=0.243
     Oracle gap (RL):   +0.063

Doc 44 (id=E06-1015) | #sentences=35
--------------------------------------------------------------------------------
  -> Challenge
     Predicted: [2, 9, 14, 16]
     Gold:      [1, 2, 8, 9, 33]
     Metrics:   P=0.50 R=0.40 F1=0.44
     ROUGE F1 (model):  R1=0.207  R2=0.090  RL=0.119
     ROUGE F1 (oracle): R1=0.466 R2=0.338 RL=0.466
     Oracle gap (RL):   +0.347
  -> Approach 
     Predicted: [3, 8, 10, 19, 20, 22]
     Gold:      [3, 19]
     Metrics:   P=0.33 R=1.00 F1=0.50
     ROUGE F1 (model):  R1=0.157  R2=0.084  RL=0.120
     ROUGE F1 (oracle): R1=0.483 R2=0.321 RL=0.448
     Oracle gap (RL):   +0.328
  -> Outcome  
     Predicted: [3, 4, 25, 26]
     Gold

In [87]:
def rename_capped_techniques(df_summary_capped, techniques_capped_str):
    """
    Replace the 'technique' column in df_summary_capped with the provided names,
    in the given order.
    """
    if len(df_summary_capped) != len(techniques_capped_str):
        raise ValueError("Mismatch: dataframe rows != techniques_capped_str length")

    df_summary_capped = df_summary_capped.copy()
    df_summary_capped["technique"] = techniques_capped_str
    return df_summary_capped

# Apply it
df_summary_capped = rename_capped_techniques(df_summary_capped, techniques_capped_str)

In [88]:
challenge_df, approach_df, outcome_df, union_df, overall_df = make_three_aspect_tables(
    df_summary_capped,
    include_metrics=["precision","recall","f1","rouge1_model","rouge2_model","rougeL_model",
                     "rouge1_oracle","rouge2_oracle","rougeL_oracle","oracle_gap_rougeL"],
    multi_model_columns=False,
)


/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x7a6b90b2c9a0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .aggregate(agg_fn))
/tmp/ipython-input-2363513484.py:65: FutureWarning: The provided callable <function mean at 0x

In [89]:
challenge_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.410833,0.373333,0.383333,0.153333,0.165000,0.205000,0.080000,0.146667,0.390833
recall,0.429075,0.373139,0.410083,0.172782,0.181571,0.223948,0.073591,0.137615,0.424452
f1,0.393655,0.352212,0.374511,0.147688,0.160644,0.198815,0.070115,0.129668,0.380553
oracle_gap_rougeL,0.216771,0.223274,0.216991,0.257669,0.305533,0.249413,0.348206,0.266789,0.222118
rouge1_model,0.212096,0.214079,0.213991,0.167801,0.090052,0.177821,0.032477,0.160207,0.210733
rouge1_oracle,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003,0.422003
rouge2_model,0.082977,0.075727,0.081090,0.038653,0.032747,0.045116,0.011457,0.035861,0.077588
rouge2_oracle,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200,0.226200
rougeL_model,0.154655,0.148152,0.154435,0.113757,0.065893,0.122013,0.023220,0.104636,0.149308


In [90]:
approach_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.351333,0.432167,0.401333,0.277333,0.129667,0.258500,0.040000,0.260833,0.425833
recall,0.358659,0.376337,0.422357,0.235960,0.124246,0.227248,0.052024,0.214726,0.449742
f1,0.337170,0.372025,0.394158,0.237729,0.121447,0.226420,0.043472,0.224156,0.418228
oracle_gap_rougeL,0.336589,0.277742,0.313221,0.303690,0.404424,0.309847,0.429420,0.301364,0.307275
rouge1_model,0.141066,0.222829,0.173748,0.200106,0.052476,0.192479,0.019587,0.203431,0.184295
rouge1_oracle,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775,0.489775
rouge2_model,0.066921,0.094299,0.083262,0.067767,0.027843,0.062815,0.010811,0.071130,0.087946
rouge2_oracle,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252,0.293252
rougeL_model,0.109590,0.168436,0.132958,0.142489,0.041754,0.136331,0.016759,0.144815,0.138904


In [91]:
outcome_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.559167,0.426667,0.524167,0.308333,0.335833,0.353333,0.135833,0.299167,0.596667
recall,0.511060,0.342628,0.504342,0.180484,0.293588,0.233767,0.135119,0.213004,0.553076
f1,0.506362,0.360269,0.490932,0.215066,0.298855,0.267911,0.130332,0.233342,0.548587
oracle_gap_rougeL,0.247844,0.238061,0.242889,0.256275,0.312965,0.248884,0.378843,0.266258,0.229012
rouge1_model,0.240768,0.262510,0.252993,0.239748,0.155816,0.249422,0.063693,0.228163,0.263351
rouge1_oracle,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727,0.496727
rouge2_model,0.106594,0.105299,0.111992,0.070868,0.071026,0.086095,0.026330,0.077378,0.119342
rouge2_oracle,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506,0.265506
rougeL_model,0.180627,0.190410,0.185583,0.172196,0.115507,0.179588,0.049629,0.162213,0.199460


As explained before, capping the results, therefore artificially preventing the model from overpredicting, leads to much better results, especially in terms of precision. Still, it seems like the outcome aspect remains the easiest to predict.

In [92]:
union_df

technique,least_to_most_capped,least_to_most_one_shot_capped,scoring_based_capped,scoring_based_one_shot_capped,self_ask_capped,self_ask_one_shot_capped,tool_augmented_capped,tool_augmented_one_shot_capped,vanilla_capped
metric,,,,,,,,,
precision,0.592439,0.594192,0.613121,0.448520,0.375755,0.481420,0.206680,0.451312,0.594780
recall,0.465985,0.481972,0.503576,0.345275,0.203722,0.380101,0.107553,0.356773,0.526226
f1,0.504292,0.521889,0.539104,0.382113,0.249661,0.416790,0.133406,0.388942,0.547217
oracle_gap_rougeL,0.184524,0.177093,0.178911,0.194128,0.241814,0.192495,0.291359,0.196662,0.188694
rouge1_model,0.285929,0.300317,0.296613,0.286192,0.201119,0.284603,0.107527,0.279963,0.278471
rouge1_oracle,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235,0.457235
rouge2_model,0.117021,0.127263,0.126271,0.100253,0.076926,0.103656,0.037277,0.104218,0.119732
rouge2_oracle,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569,0.229569
rougeL_model,0.171080,0.178511,0.176692,0.161475,0.113789,0.163108,0.064244,0.158941,0.166909


The union dataframe, which represents the outcomes toghether, uniting the gold standard and the prediction, gives ulterior credit to this hypothesis.

### Qwen

In [58]:
import os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_qwen_fast(model_name="Qwen/Qwen3-4B"):
    # perf knobs
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision("high")
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=True)
    tok.padding_side = "left"

    # Try FlashAttention 2 first
    attn_impl = "flash_attention_2"
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            attn_implementation=attn_impl,
            device_map="cuda",
        )
        print("Qwen loaded with FlashAttention 2")
    except Exception as e:
        print(f"FA2 not available ({e}); falling back to SDPA.")
        attn_impl = "sdpa"
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            attn_implementation=attn_impl,
            device_map="cuda",
        )

    model.generation_config.pad_token_id = tok.pad_token_id or tok.eos_token_id
    model.generation_config.eos_token_id = tok.eos_token_id
    model.config.use_cache = True
    model.eval()
    return model, tok, attn_impl


In [60]:
model_name = "Qwen/Qwen3-4B" # Or "meta-llama/Llama-3.2-3B-Instruct" || Qwen/Qwen3-4B

model, tokenizer, attn_impl = load_qwen_fast(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Qwen loaded with FlashAttention 2


In [61]:
results_qwen = []
for tech in techniques:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=False,
        oracle_max_len=None,
        capped = False
    )
    results_qwen.append(res)

df_docs_qwen, df_summary_qwen = results_to_dataframe(results_qwen)

Model Name: Qwen/Qwen3-4B | Technique: <function vanilla_prompt at 0x7a69c50402c0>
Model Name: Qwen/Qwen3-4B | Technique: <function least_to_most_prompt at 0x7a69c5040a40>
Model Name: Qwen/Qwen3-4B | Technique: <function least_to_most_one_shot at 0x7a69c5041300>
Model Name: Qwen/Qwen3-4B | Technique: <function tool_augmented_prompt at 0x7a69c5041c60>
Model Name: Qwen/Qwen3-4B | Technique: <function tool_augmented_one_shot at 0x7a69c50425c0>


KeyboardInterrupt: 

In [ ]:
df_summary_qwen

In [ ]:
results_qwen_capped = []
for tech in techniques:
    print(f"Model Name: {model_name} | Technique: {tech}")
    res = run_evaluation_with_rouge_and_oracle(
        dataset=dataset,
        model_name=model_name,
        model=model,
        tokenizer=tokenizer,
        prompt_technique=tech,
        gold_all=gold_all,
        aspects=ASPECTS,
        abs_by_id=abs_by_id,
        start=0,
        stop=None,
        verbose=False,
        oracle_max_len=None,
        capped = True
    )
    results_qwen_capped.append(res)

df_docs_qwen_capped, df_summary_qwen_capped = results_to_dataframe(results_qwen_capped)

In [ ]:
df_summary_qwen_capped